In [2]:
import pandas as pd
file_name = "D:\OneDrive - Inventec Corp\文件\IEC_DMSpec_v1.6_20240510.xlsx"
all_sheet_names = (pd.ExcelFile(file_name,engine='openpyxl')).sheet_names
all_sheet_names = all_sheet_names[4:]

for sheet_name in all_sheet_names:
    # print(sheet_name,'對應' not in sheet_name and 'mapping' not in sheet_name)
    if '對應' not in sheet_name and 'mapping' not in sheet_name and 'Mapping' not in sheet_name:
        print(sheet_name)
        table_name = pd.read_excel(file_name, sheet_name=sheet_name, header=None, nrows=1).values[0][3]
        df = pd.read_excel(file_name, sheet_name=sheet_name, header=8)
        df = df.drop(0, axis=0).drop(df.columns[0], axis=1)
        sql_query = f"""USE [BSDB] \nGO \nSET ANSI_NULLS ON \nGO \nSET QUOTED_IDENTIFIER ON \nGO \nCREATE TABLE STAGE.{table_name} (\n"""
        PK_SQL = f" CONSTRAINT [PK_STAGE_{table_name}] PRIMARY KEY CLUSTERED \n ( \n" 
        for column, dtype, length, PK, NN in zip(
            df['Field Name'], df['Fmt.'], df['Length'], df['PK'], df['NN']):
            if dtype == 'nvarchar':
                if length=='Max':
                    sql_type = f"{dtype}({length})"
                else:
                    sql_type = f"{dtype}({int(length)})"
            elif dtype == 'numeric':
                sql_type = f"{dtype}(11,2)"
            elif dtype == 'DateTime':
                sql_type = f"{dtype}"

            if NN == "V":
                sql_query += f"    {column} {sql_type} Not Null,\n "
            else:
                sql_query += f"    {column} {sql_type} Null,\n "

            if PK == "V":
                PK_SQL += f"    [{column}] ASC,\n"
            # 删除最后一个逗号，并添加结束括号
        pk_sql_query = PK_SQL[:-2] + "\n)"+ f"""WITH (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, IGNORE_DUP_KEY = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY]
        ) ON [PRIMARY]
        GO
        CREATE VIEW [ODS].{table_name}
AS Select * from STAGE.{table_name}
; 
  
GO"""
        sql_query += pk_sql_query
        # 打印 SQL 语句
        print(sql_query)
    # 将 SQL 查询语句写入到文本文件中
    with open(f'D:/Budget System/SQL_Server/{table_name}.txt', 'w') as file:
        file.write(sql_query)

月平均匯率
USE [BSDB] 
GO 
SET ANSI_NULLS ON 
GO 
SET QUOTED_IDENTIFIER ON 
GO 
CREATE TABLE STAGE.EXCHANGE_RATE (
    YEAR nvarchar(4) Not Null,
     PERIOD nvarchar(20) Not Null,
     SCENARIO nvarchar(20) Not Null,
     VERSION nvarchar(20) Not Null,
     DATA nvarchar(20) Not Null,
     CURRENCY nvarchar(20) Not Null,
     ACCOUNT nvarchar(20) Not Null,
     ENTITY nvarchar(20) Not Null,
     CUSTOMER nvarchar(20) Not Null,
     PRODUCT nvarchar(50) Not Null,
     PRODUCT_TYPE nvarchar(20) Not Null,
     SITE nvarchar(20) Not Null,
     CUSTOM1 nvarchar(50) Not Null,
     CUSTOM2 nvarchar(50) Not Null,
     CUSTOM3 nvarchar(50) Not Null,
     AMOUNT numeric(11,2) Null,
     CREATE_DATE DateTime Null,
  CONSTRAINT [PK_STAGE_EXCHANGE_RATE] PRIMARY KEY CLUSTERED 
 ( 
    [YEAR] ASC,
    [PERIOD] ASC,
    [SCENARIO] ASC,
    [VERSION] ASC,
    [DATA] ASC,
    [CURRENCY] ASC,
    [ACCOUNT] ASC,
    [ENTITY] ASC,
    [CUSTOMER] ASC,
    [PRODUCT] ASC,
    [PRODUCT_TYPE] ASC,
    [SITE] ASC,
 